In [1]:
from typing import Annotated
import os
from IPython.display import Image, display
from typing_extensions import TypedDict

from langchain_tavily import TavilySearch
from langchain.chat_models import init_chat_model
from langchain.embeddings import init_embeddings
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
# from langgraph.checkpoint.memory import InMemorySaver
# # from langgraph_checkpoints.postgres import PostgresSaver
# from langgraph.checkpoint.memory import InMemorySaver

from langgraph.graph import START, MessagesState, StateGraph
from langgraph.types import Command, interrupt
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import MessagesState
from langchain_core.runnables.config import RunnableConfig

from dotenv import load_dotenv # 把 api_key 存在環境變數中，然後用 load_dotenv() 讀取
load_dotenv()

litellm_api_key = os.getenv('LITELLM_API_KEY')
litellm_api_base = os.getenv('LITELLM_API_BASE')
tavily_search_api_key = os.getenv('TAVILY_SEARCH_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='azure-gpt-4.1',
    api_key=litellm_api_key,
    base_url=litellm_api_base,
    temperature=0.9,
    top_p=0.8,
)

In [3]:
tools = [
    TavilySearch(max_results=10,
                 tavily_api_key=tavily_search_api_key)
]

llm_with_tool = llm.bind_tools(tools)

### Langgraph Web Search

In [ ]:
def chatbot(state: MessagesState):
    # 加入 system prompt
    if not any(isinstance(msg, SystemMessage) for msg in state["messages"]):
        system_message = SystemMessage(content="""
		你是一個專業且有幫助的汽車/機車查詢助理。  
		你可以使用搜尋工具來回答問題。
                                       
		請遵守以下規則：  
		1. 你的任務是搜尋並整理 **汽車或機車車型** 的資訊。  
		2. 回答時必須提供：  
        
        - 廠牌或廠商
		- 中文名稱，不需要列出廠牌或廠商的資訊（包含中英文），有重複出現，請只回覆一次就好，不要重複列出。
		- 英文名稱，不需要列出廠牌或廠商的資訊（包含中英文），有重複出現，請只回覆一次就好，不要重複列出。
		- 常見名稱（若有多個，請列出），且每一個不需要列出廠牌或廠商的資訊（包含中英文），有重複出現，請只回覆一次就好，不要重複列出。
		
        3. 如果在中文名稱出現過的車型，英文名稱也出現過的車型，以及常見名稱也出現過的車型，請只回覆一次就好，不要重複列出。
        4.不需提供車型代碼
                                       
    	5. 請使用繁體中文作答。  
		6. 輸出請盡量用條列或結構化方式呈現，方便閱讀。  
		範例：
		車型代碼：NXC125N  
		中文名稱：勁戰三代  
		英文名稱：CYGNUS-X (3rd Gen)  
		常見名稱：勁戰 125、CYGNUS-X 125、勁戰
        
		""")
        state["messages"].insert(0, system_message)
    
    messages = llm_with_tool.invoke(state["messages"])
    return {"messages": messages}

In [8]:
graph_builder = StateGraph(state_schema=MessagesState)
tool_node = ToolNode(tools)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node('tools', tool_node)

graph_builder.add_edge(START, "chatbot") # Start the conversation with the chatbot
graph_builder.add_conditional_edges("chatbot", tools_condition) # Add conditional edges for tool usage(讓llm自行決定判斷條件)
graph_builder.add_edge("tools", "chatbot") # Allow the tools to call the chatbot 

graph = graph_builder.compile()

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system","""
		你是一個專業且有幫助的汽車/機車查詢助理。  
		你可以使用搜尋工具來回答問題。
                                       
		請遵守以下規則：  
		1. 你的任務是搜尋並整理 **汽車或機車車型** 的資訊。  
		2. 回答時必須提供：  
        
        - 廠牌或廠商
		- 中文名稱，不需要列出廠牌或廠商的資訊（包含中英文），有重複出現，請只回覆一次就好，不要重複列出。
		- 英文名稱，不需要列出廠牌或廠商的資訊（包含中英文），有重複出現，請只回覆一次就好，不要重複列出。
		- 常見名稱（若有多個，請列出），且每一個不需要列出廠牌或廠商的資訊（包含中英文），有重複出現，請只回覆一次就好，不要重複列出。
		
        3. 如果在中文名稱出現過的車型，英文名稱也出現過的車型，以及常見名稱也出現過的車型，請只回覆一次就好，不要重複列出。
        4.不需提供車型代碼
                                       
    	5. 請使用繁體中文作答。  
		6. 輸出請盡量用條列或結構化方式呈現，方便閱讀。  
		範例：
		車型代碼：NXC125N  
		中文名稱：勁戰三代  
		英文名稱：CYGNUS-X (3rd Gen)  
		常見名稱：勁戰 125、CYGNUS-X 125、勁戰
        
		"""),
        ("user", "NXC125N 的車型是什麼，上網幫我搜尋一下")        
])
chain = prompt | llm
response = chain.invoke({})
print(response.content)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 37-38: ordinal not in range(128)

In [ ]:
from langchain_core.messages import HumanMessage

user_input = "NXC125N 的車型是什麼，上網幫我搜尋一下"
state = MessagesState(messages=[HumanMessage(content=user_input)])

config = {"configurable": {"thread_id": "1", "user_id": "1"}}

response = graph.invoke(state, config=config)

# llm 的回覆（response 會是歷史所有資料，最後一個是 llm 的回覆）
print(response['messages'][-1].content)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 37-38: ordinal not in range(128)

### 結構化輸出

In [7]:
from typing import Optional, List
from pydantic import BaseModel, Field 

# BaseModel：Pydantic 提供的基底類別，所有模型都繼承它。
# Field(...)：用來設定欄位的描述、預設值、驗證條件等等
class VehicleInfo(BaseModel): # 繼承 BaseModel
    # query: Optional[str] = Field(None, description="查詢的車型，例如 GQR125CD") # 可以為 str 或 None
    chinese_name: Optional[str] = Field(None, description="中文車名")
    english_name: Optional[str] = Field(None, description="英文車名")
    common_names: List[str] = Field(default_factory=list, description="常見名稱清單，可能有多個") # 定義成 list ，因為會有多個常見名稱
    # cc: Optional[int] = Field(None, description="排氣量，若搜尋不到請在這裏先跟我說搜尋不到，再回覆最接近的排氣量")
    # othertypes: List[str] = Field(default_factory=list, description="其他類型或版本清單，可能有多個")

structured_llm = llm.with_structured_output(VehicleInfo)


In [8]:
structured_llm.invoke(response['messages'][-1].content).model_dump()

{'chinese_name': '勁戰三代',
 'english_name': 'CYGNUS-X (3rd Gen)',
 'common_names': ['勁戰 125', 'CYGNUS-X 125', '勁戰']}

In [12]:
import pandas as pd

df_cnt = pd.read_csv('../car_kind_cc_count_new.csv')

In [13]:
df_cnt

,CAR_KIND,ENGINE_EXHAUST,count,SYS_NO,CAR_TYPE,STATUS,VEHICLE_KIND,FUEL,CAR_MODEL,MADE_YEAR_MONTH,CAR_COLOR,VEHICLE_KIND_ALIAS,SEAT_CNT
0,ZE1EPE,1794 cc,14484,AA,汽車,TRUE,自用小客車,汽油,國瑞,200501,白,沒有查詢,沒有查詢
1,TOYOTA RAV4,1987 cc,13724,AT,汽車,TRUE,自用小客車,汽油,TOYOTA,202001,白,03-自用小客車,5
2,ZRE142L-GEXEKR,1798 cc,11897,AA,汽車,TRUE,自用小客車,汽油,國瑞,201107,白,NaN,NaN
3,ZRE172L-GEXGKR,1798 cc,11328,AT,汽車,TRUE,自用小客車,汽油,國瑞,201609,灰,03-自用小客車,5
4,ZRE172L-GEXEKR,1798 cc,10914,AA,汽車,TRUE,自用小客車,汽油,國瑞,201805,白,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27524,.LEXUS RX350,3456 cc,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27525,作4和4 AVANT 3.0T,1984 cc,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27526,作A12T工,124 cc,1,AT,機車,TRUE,普通重型,汽油,三陽,199804,綜,NaN,NaN
27527,化52.0 文,1997 cc,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### for loop 查詢
1. 透過 web search graph 查詢結果
2. 透過 structured_llm 結構化輸出

In [ ]:
# web search graph 查詢
from typing import Dict, Any
def build_car_kind_search_prompt(row: Dict[str, Any]) -> str:
    return f"幫我搜尋以下車子的車型資訊：\n- 廠牌：{row['CAR_MODEL']} - 車型：{row['CAR_KIND']}"


sub_df = df_cnt.head(10)
result = []
for i, row in sub_df.iterrows():
    
    prompt = build_car_kind_search_prompt(row)
    resp = graph.invoke(MessagesState(messages=[prompt]), config=config)
    resp = resp['messages'][-1].content
    result.append(resp)
    
sub_df['llm_result'] = result

In [ ]:
structured_result = []
for i, row in sub_df.iterrows():
    resp = structured_llm.invoke(row['llm_result'])
    structured_result.append(resp.model_dump())
    
result_df = pd.DataFrame(structured_result)
result_df = result_df.fillna('')

result_df['chinese_name'] = result_df.chinese_name.apply(lambda x: [x])
result_df['english_name'] = result_df.english_name.apply(lambda x: [x])
result_df['alias_kind_name'] = result_df['chinese_name'] + result_df['english_name'] + result_df['common_names']
result_df['alias_kind_name'] = result_df['alias_kind_name'].apply(lambda x: list(set(x)))

In [ ]:
final_df = pd.concat([sub_df, result_df], axis=1)